# Renaming Abstracts

After examining around 5000 abstracts manually I found the following:

Empty abstracts - nothing we can do about that

Abstract containing only "None", "unk", "Not avail",  etc. --> define a min length for abstract, otherwise erase it and make an empty abstract

Abstract starting with "INTRODUCTION:" "what you need to know", "OBJECTIVE:", "OBJECTIVES", "Objective", "BACKGROUND:", "PURPOSE", "License:", "Abstract:", "Abstract: ", "ABSTRACT:" -->these words are predictable so we can simply delete them

Abstract starting with "&lt:p&gt:&", "10.1186/1471-25" --> delete the first work if it has too many numbers or non letters characters

Abstracts that repeat the title --> empty the abstract

Abstracts that finish with "..." --> remove the ...

Non alphanumerical characters --> not sure hoe to solve this

Chinese characters --> not sure how to solve this

**Overall need to do some code to better understand these abstracts** --> this is what this code is for


## Results

**empty Abstracts** are Np.NaN (less than 60 chararcters are also NaN, with a few exeptions).



## Still TODO (relatively simple)

Abstract starting with "INTRODUCTION:" "what you need to know", "OBJECTIVE:", "OBJECTIVES", "Objective", "BACKGROUND:", "PURPOSE", "License:", "Abstract:", "Abstract: ", "ABSTRACT:", "Summary" -->these words are predictable so we can simply delete them (I just want to see if there is any other words like this preceding the abstracts)

Abstract starting with "&lt:p&gt:&", "10.1186/1471-25" --> delete the first work if it has too many numbers or non letters characters

Abstract that talk about the availability and website etc. Want to delete this part from the abstract

In [1]:
from os.path import join as ospj
import os
import sys
import pandas as pd
import numpy as np
import sys
print(sys.version)
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import math

3.6.10 (default, Dec 19 2019, 23:04:32) 
[GCC 5.4.0 20160609]


In [2]:
import re

In [3]:
def clean_and_tokenize(text):
    """
    Cleaning a document with:
        - Lowercase        
        - Removing numbers with regular expressions
        - Removing punctuation with regular expressions
        - Removing other artifacts
    And separate the document into words by simply splitting at spaces
    Params:
        text (string): a sentence or a document
    Returns:
        tokens (list of strings): the list of tokens (word units) forming the document
    """        
    # Lowercase
    text = text.lower()
    # Remove numbers
    text = re.sub(r"[0-9]+", "", text)
    # Remove punctuation
    REMOVE_PUNCT = re.compile("[.;:!\'?,\"()\[\]]")
    text = REMOVE_PUNCT.sub("", text)
    # Remove HTML artifacts specific to the corpus we're going to work with
    REPLACE_HTML = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
    text = REPLACE_HTML.sub(" ", text)
    
    tokens = text.split()        
    return tokens

In [4]:
pd.set_option('display.max_rows', 25)

data_path = '/home/marchaln/CodeVsCovid19/data/metadata.csv'
data_path = '../data'

metadata_path = ospj(data_path, 'metadata_complet.csv')
new_metadata_path = ospj(data_path, 'metadata_new_new.csv')

metadata_pd = pd.read_csv(metadata_path)  

In [5]:
#keep track of the number of changes we do
tot_changes = 0

In [6]:
#delete abstracts with less than 30 chararcters

df = metadata_pd

for i, row in df.iterrows():
    one_abs = df['abstract'][i]
    if len(str(one_abs)) < 30:
        df.at[i,'abstract']=np.NaN
        
        if not pd.isnull(one_abs) :
            tot_changes +=1 
            if one_abs != 'Unknown'and one_abs != '[Image: see text]':
                print(str(one_abs))
                
print('number of changes :', tot_changes )

None
Wash hands thoroughly.
Not Avail.
A Foresight study
While …
Abstract
Abstract No Abstract
[Figure: see text]
Set by Richard Smyth
number of changes : 351


In [7]:
# this is used to find the abstract of less than 60 chararcters not to delete

exceptions = ['Approvals, new indications, regulatory activities, and more',
             'Director general of the World Health Organization',
             'PR chief says they must learn how to present their stories',
             'Strategic risk communication for new and emerging risks',
             'Fast action will help the world contain the new coronavirus',
             'Sorting the week\'s supernovae from the absolute zeros',
             'Many drugs and vaccines are now being developed and tested']

tot_changes_prior = tot_changes 

for i, row in df.iterrows():
    one_abs = df['abstract'][i]
    if len(str(one_abs)) < 60 and not pd.isnull(one_abs) :
        
        
        if str(one_abs) in exceptions:
            print('NOT DELETED')
            print(str(df['title'][i]))
            print(str(one_abs)+ '\n')
            
        else:
            print('DELETED')
            print(str(df['title'][i]))
            print(str(one_abs)+ '\n')    
            df.at[i,'abstract']=np.NaN 
            tot_changes += 1
            
print('number of changes :', tot_changes - tot_changes_prior )

DELETED
Resource Allocation during an Influenza Pandemic
Resource Allocation during an Influenza Pandemic

DELETED
Coronaviruses: Molecular and Cellular Biology
Coronaviruses: Molecular and Cellular Biology

DELETED
Isolation of Novel Adenovirus from Fruit Bat (Pteropus dasymallus yayeyamae)
Isolation of Novel Adenovirus from Fruit Bat

DELETED
Manual of Travel Medicine and Health
Manual of Travel Medicine and Health

DELETED
Ethical Alternatives to Experiments with Novel Potential Pandemic Pathogens
Please see later in the article for the Editors' Summary

NOT DELETED
Lee Jong-wook
Director general of the World Health Organization

NOT DELETED
Drug and Device News
Approvals, new indications, regulatory activities, and more

DELETED
Flu: A Medical Mystery
BBC Radio 4, 2 February at 8 pm Rating: ★★★

NOT DELETED
Public health doctors "hopeless" at using media
PR chief says they must learn how to present their stories

NOT DELETED
When even the 'best-laid' plans go wrong
Strategic risk c

In [8]:
#delete when the abstract is a duplicate of the title

tot_changes_prior = tot_changes 

for i, row in df.iterrows():
    one_abs = df['abstract'][i]
    one_title = df['title'][i]
    diff = len(str(one_abs)) - len(str(one_title))
    diff = math.fabs(int(diff))
    
    if not pd.isnull(one_abs):
        #it is not always a pure duplicate but there is no more information than alredy in the title
        if (one_abs in one_title) or (one_title in one_abs) and diff < 200:
            if (one_abs[-3:] != '...')  :
                if (one_abs[-1:] != "\u2026") :
                    print('DELETE similar title and abstract:')
                    print('TITLE')
                    print(one_title)
                    print('ABSTRACT')
                    print(one_abs)
                    print('\n')
                    df.at[i,'abstract']=np.NaN 
                    tot_changes += 1
                else:
                    print('KEEP similar title and abstract:')
                    print('TITLE')
                    print(one_title)
                    print('ABSTRACT')
                    print(one_abs)
                    print('\n')
            else:
                print('KEEP similar title and abstract:')
                print('TITLE')
                print(one_title)
                print('ABSTRACT')
                print(one_abs)
                print('\n')


print('number of changes :', tot_changes - tot_changes_prior )

DELETE similar title and abstract:
TITLE
Stability of SARS-CoV-2 in different environmental conditions
ABSTRACT
Stability of SARS-CoV-2 in different environmental conditions.


DELETE similar title and abstract:
TITLE
Statistics-Based Predictions of Coronavirus Epidemic Spreading in Mainland China
ABSTRACT
Information about the open-access article 'Statistics-Based Predictions of Coronavirus Epidemic Spreading in Mainland China' in DOAJ. DOAJ is an online directory that indexes and provides access to quality open access, peer-reviewed journals.


DELETE similar title and abstract:
TITLE
Wuhan novel coronavirus 2019nCoV
ABSTRACT
Information about the open-access article 'Wuhan novel coronavirus 2019nCoV' in DOAJ. DOAJ is an online directory that indexes and provides access to quality open access, peer-reviewed journals.


KEEP similar title and abstract:
TITLE
Virtually Perfect? Telemedicine for Covid-19
ABSTRACT
Virtually Perfect? Telemedicine for Covid-19 Telemedicine’s payment and re

In [9]:
#find out the number of articles for which the abstract ends with "..." :

tot_changes_prior = tot_changes 

for i, row in df.iterrows():
    one_abs = df['abstract'][i]
    one_title = df['title'][i]
    if not pd.isnull(one_abs):
        if one_abs[-3:] == '...' :
            print(one_abs)
            print('\n')
            df.at[i,'abstract']= one_abs[:-3]
            tot_changes += 1
        if one_abs[-1:] ==  "\u2026":
            print(one_abs)
            print('\n')
            df.at[i,'abstract']= one_abs[:-1]
            tot_changes += 1
            
print('number of changes :', tot_changes - tot_changes_prior )
            
#That turns out there are 20 of them but it is not a problem because the abstract 
# are long enough to uderstand what the paper talks about (sort of)
# however we must delete the ... because nlp might use this to think that these abstracts
# have something in common

Virtually Perfect? Telemedicine for Covid-19 Telemedicine’s payment and regulatory structures, licensing, credentialing, and implementation take time to work through, but health systems that have a...


Now calm down, we’re not about to start dolling out fashion tips for your facial fuzz. We’re talking about how having a hairy face could stop protective face masks from fitting properly.It can do. Derek Sandeman, the medical director of the University Hospital Southampton NHS Foundation Trust, has written to his clinical staff asking anyone working in high risk …


Healthcare staff on leave and those who have retired could be called “back to duty,” and non-urgent care could be delayed, as doctors are forced to prioritise dealing with covid-19, the UK government’s action plan lays out.The government will also implement a “distribution strategy for the UK’s stockpiles of key medicines and equipment (e.g. protective clothing),” the document said.However, the plan did not include details of

Infectious diseases are caused by infectious organisms. Typically, these are bacteria, …


The influenza viruses, which contain single-stranded RNA, are classified into 3 types, A, B, and C …


Bronchitis is characterized by bronchial inflammation that results in …


number of changes : 20


In [10]:
#Abstract starting with 
#a space or "/"
# "INTRODUCTION:" 
# "what you need to know", 
# "OBJECTIVE: ", "OBJECTIVE ", "OBJECTIVE. ", "Objective", "OBJECTIVE(S): ", "OBJECTIVES AND DESIGN: ", "OBJECTIVE--"
# "OBJECTIVES: ", "OBJECTIVES. ", "OBJECTIVES "
#  "BACKGROUND: ", "BACKGROUND ", "BACKGROUND"
# "PURPOSE: ", "PURPOSE OF REVIEW: ", "PURPOSE OF THE REVIEW: ", "PURPOSE AND EXPERIMENTAL DESIGN: ", "PURPOSE ", "Purpose. "
# "Abstract:", "Abstract: ", "ABSTRACT:",
#  "Summary",  "Summary ",  "Summary: ", "Summary / Abstract", 

# "motivation", "importance", "overview", "aim", " Rationale", "Objetivo: ", "Propósito: ",
# "S", "Context", "[Image: see text]", "-: "Conclusion", "Methods", "teaser", "issue"

# -->these words are predictable so we can simply delete them 
#(I just want to see if there is any other words like this preceding the abstracts)



def remove_beginning(df, i, keyword):
    
    count = 0
    length = len(keyword)
    
    one_abs = df['abstract'][i]
    while one_abs[0] == " " or one_abs[0] == "/":
        one_abs = one_abs[1:]
        df.at[i,'abstract']= one_abs
        
    if one_abs[:length+3].lower() == keyword + " / ":    
        count += 1
        df.at[i,'abstract']= one_abs[length+3:]
    elif one_abs[:length+4].lower() == keyword + "s / ":    
        count += 1
        df.at[i,'abstract']= one_abs[length+4:]
    elif one_abs[:length+1].lower() == keyword + "/":    
        count += 1
        df.at[i,'abstract']= one_abs[length+1:]
    elif one_abs[:length+2].lower() == keyword + "s/":    
        count += 1
        df.at[i,'abstract']= one_abs[length+2:]
    elif one_abs[:length+3].lower() == keyword + " : ":    
        count += 1
        df.at[i,'abstract']= one_abs[length+3:]
    elif one_abs[:length+2].lower() == keyword + ": ":    
        count += 1
        df.at[i,'abstract']= one_abs[length+2:]
    elif one_abs[:length+2].lower() == keyword + ". ":    
        count += 1
        df.at[i,'abstract']= one_abs[length+2:]
    elif one_abs[:length+1].lower() == keyword + " ":    
        count += 1
        df.at[i,'abstract']= one_abs[length+1:]
    # plurals
    elif one_abs[:length+4].lower() == keyword + "s : ":    
        count += 1
        df.at[i,'abstract']= one_abs[length+4:]
    elif one_abs[:length+3].lower() == keyword + "s: ":    
        count += 1
        df.at[i,'abstract']= one_abs[length+3:]
    elif one_abs[:length+2].lower() == keyword + ". ":    
        count += 1
        df.at[i,'abstract']= one_abs[length+2:]
    elif one_abs[:length+2].lower() == keyword + "s ":    
        count += 1
        df.at[i,'abstract']= one_abs[length+2:]
    elif one_abs[:length+1].lower() == keyword + "s":    
        count += 1
        df.at[i,'abstract']= one_abs[length+1:]
    elif one_abs[:length+0].lower() == keyword:    
        count += 1
        df.at[i,'abstract']= one_abs[length+0:]
        
    return count, df

    
    
remove_words = ['summary', 'introduction', 'abstract', 'background', 'purpose', 'aim', 'importance', 
                'objective', 'introduction', 'propósito', 'objetivo', 'rationale', 'motivation', 'context',
               'issue', 'hypotheses', 'problem', 'setting', 'presentation', 'method', 'overview', 'conclusion',
               'significance', 'teaser', 'résumé' ]
        
tot_count = np.zeros(len(remove_words))
OTHER_count = 0

for i, row in df.iterrows():
    one_abs = df['abstract'][i]
    one_title = df['title'][i]
    if not pd.isnull(one_abs):
                       
        #exceptions
        one_abs = df['abstract'][i]
        while one_abs[0] == " " or one_abs[0] == "/":
            one_abs = one_abs[1:]
            df.at[i,'abstract']= one_abs
        if one_abs[:21] == "What you need to know": # "What you need to know"    
            OTHER_count += 1
            df.at[i,'abstract']= one_abs[21:]
        elif one_abs[:11].lower() == "backgound: ":  #spelling mistake  
            OTHER_count += 1
            df.at[i,'abstract']= one_abs[11:]
        elif one_abs[:11].lower() == "backgroud: ":  #spelling mistake  
            OTHER_count += 1
            df.at[i,'abstract']= one_abs[11:]          
        elif one_abs[:25].lower() == "importance of the field: ":    
            OTHER_count += 1
            df.at[i,'abstract']= one_abs[25]            
        elif one_abs[:23].lower() == "objectives and design: ":
            OTHER_count += 1
            df.at[i,'abstract']= one_abs[23:]         
        elif one_abs[:14].lower() == 'objective(s): ': 
            OTHER_count += 1
            df.at[i,'abstract']= one_abs[14:]
        elif one_abs[:11].lower() == "objectives—":
            OTHER_count += 1
            df.at[i,'abstract']= one_abs[11:]  
        # [Image: see text]
        elif one_abs[:18].lower() == "[image: see text] ":    
            OTHER_count += 1
            df.at[i,'abstract']= one_abs[18:]  
        # [Table: see text] 
        elif one_abs[:18].lower() == "[table: see text] ":    
            OTHER_count += 1
            df.at[i,'abstract']= one_abs[18:] 
        elif one_abs[:23].lower() == "1summary 1.1 background":    
            OTHER_count += 1
            df.at[i,'abstract']= one_abs[23:]  
             
            
            
        # remove all the words
        for j, word in enumerate(remove_words):
            count, df = remove_beginning(df, i, word)
            tot_count[j] += count
                
            
        #final cleaning
        one_abs = df['abstract'][i]
        while one_abs[0] == " " or one_abs[0] == "/" or one_abs[0] == "-" or \
            one_abs[0] == ":" or one_abs[0] == "." :
            one_abs = one_abs[1:]
            df.at[i,'abstract']= one_abs
        
        #display the rest
        one_abs = df['abstract'][i]
        if one_abs.split()[0][-1] == ":":    
            #pass
            print(one_abs + '\n')
            #SUMMARY_count += 1
            #df.at[i,'abstract']= one_abs[7:]
        

for j, word in enumerate(remove_words):
    print(word, tot_count[j])        
print('\nother', OTHER_count)            



tot_changes +=  np.sum(tot_count) + OTHER_count


It: Two Coronaviruses, HCoV-229E and HCoV-OC43, causing generally mild respiratory tract infections in humans, were described in the XX c. Pandemic Coronaviruses were first discovered as late as in the XXI c.: SARS-HCoV in 2002 – causing severe respiratory tract infections (SARS) in China; MERS-HCoV in 2012 – circulating mostly on the Arabian Peninsula. The SARS epidemic ended in 2004 resulting in morbidity of >8000 and >770 deaths, while the MERS epidemic is still ongoing (>2000 ill, >700 deaths) although its intensity decreased. Both viruses are zoonotic and require at least two “host jumps” for the transmission of the infection to humans: for HCoV-SARS – from bat to palm civet and then to human; for HCoV-MERS – from bats to camels and subsequently to humans. Primary mode of transmission is droplet in close contact (<1 m), but both viruses remain active in aerosol (up to 24 h), so infection can be also spread by air (ventilation). The ability for human-to-human transmission is higher

In [11]:
workd = 'OBJECTIVE(S): '
print(workd.lower())

if 'objective(s): ' == workd.lower()[:14]:
    print('ok')

objective(s): 
ok


In [12]:
print('Total number of changes :', tot_changes)

Total number of changes : 17884.0


In [13]:
#save the new dataframe
if os.path.isfile(new_metadata_path):
    sys.exit('ERROR, a new dataset is already saved')
else:
    df.to_csv(new_metadata_path)